## Sawtooth demo

In [ ]:
# setup logging

import elfpy.utils.outputs as output_utils

output_utils.setup_logging(log_filename="../../.logging/sawtooth_demo.log")

In [ ]:
# setup custom agent

from elfpy.types import MarketActionType
from elfpy.agent import Agent

class Shorter(Agent):
    """
    Agent that is trying to optimize on a rising vault APR via shorts
    """

    def __init__(self, wallet_address, budget=10_000, pt_to_short=1_000):
        """call basic policy init then add custom stuff"""
        self.pt_to_short = pt_to_short
        super().__init__(wallet_address, budget)

    def action(self, market, pricing_model):
        """
        implement user strategy
        short if you can, only once
        """
        block_position_list = list(self.wallet.shorts.values())
        has_opened_short = bool(any((x < -1 for x in block_position_list)))
        can_open_short = self.get_max_pt_short(market, pricing_model) >= self.pt_to_short
        current_vault_apr = market.market_state.vault_apr
        current_pool_apr = market.get_rate(pricing_model)
        action_list = []
        if can_open_short:
            if current_vault_apr > current_pool_apr:
                action_list.append(self.create_agent_action(
                    action_type=MarketActionType.OPEN_SHORT, trade_amount=self.pt_to_short)
                )
            elif current_vault_apr < current_pool_apr:
                if has_opened_short:
                    action_list.append(
                        self.create_agent_action(action_type=MarketActionType.CLOSE_SHORT, trade_amount=self.pt_to_short)
                    )
        return action_list

In [ ]:
import numpy as np

from elfpy.utils.parse_config import parse_simulation_config
from elfpy.simulators import Simulator

In [ ]:
config = config_dict={
    'title': 'demo simulation config',
    'market': {
        'min_target_liquidity': 1_000_000.0,
        'max_target_liquidity': 10_000_000.0,
        'min_vault_age': 0,
        'max_vault_age': 1,
        'vault_apr': {"type": "uniform", "low": 0.001, "high": 0.9},
        'base_asset_price': 1
    },
    'amm': {
        'pricing_model_name': 'Hyperdrive',
        'min_fee': 0.1,
        'max_fee': 0.5,
        'min_pool_apr': 0.02,
        'max_pool_apr': 0.9,
        'floor_fee': 0,
    },
    'simulator': {
        'num_trading_days': 10,#180, # TODO: overriding this is broken
        'num_blocks_per_day': 7_200,
        'token_duration': 0.2465753424657534,
        'precision': 64,
        'agent_policies': [],
        'shuffle_users': True,
        'init_lp': True,
        'random_seed': 123,
        'logging_level': 'debug'
    }
}

In [ ]:
ramp = np.linspace(start=0.05, stop=0.1, num=config['simulator']['num_trading_days'])
override_dict = {
    "target_liquidity": 1e7,
    "fee_percent": 0.1,
    "target_pool_apr": 0.05,
    "vault_apr": ramp.tolist(),
    "num_trading_days": 10,
    "num_blocks_per_day": 1,  # temporarily set it to 1 block a day for testing
}

In [ ]:
from elfpy.utils import sim_utils

# instantiate config object
config = sim_utils.override_config_variables(parse_simulation_config(config), override_dict)
# instantiate random number generator
rng = np.random.default_rng(config.simulator.random_seed)
# run random number generators to get random simulation arguments
random_sim_vars = sim_utils.override_random_variables(
    sim_utils.get_random_variables(config, rng), override_dict
)
# instantiate the pricing model
pricing_model = sim_utils.get_pricing_model(model_name=config.amm.pricing_model_name)
# instantiate the market
market = sim_utils.get_market(
    pricing_model,
    random_sim_vars.target_pool_apr,
    random_sim_vars.fee_percent,
    config.simulator.token_duration,
    random_sim_vars.vault_apr,
    random_sim_vars.init_share_price,
)
# instantiate the init_lp agent
init_agents = {
    0: sim_utils.get_init_lp_agent(
        market,
        pricing_model,
        random_sim_vars.target_liquidity,
        random_sim_vars.target_pool_apr,
        random_sim_vars.fee_percent,
    )
}
# set up simulator with only the init_lp_agent
simulator = Simulator(
    config=config,
    pricing_model=pricing_model,
    market=market,
    agents=init_agents,
    rng=rng,
    random_simulation_variables=random_sim_vars,
)
simulator.collect_and_execute_trades()

In [ ]:
# initialize the market using the LP agent
num_additional_agents = 5
budget = 1_000_000_000
pt_to_short = 500_000
for agent_address in range(1, num_additional_agents+1):
    simulator.agents.update(
        {
            agent_address: Shorter(
                wallet_address=agent_address,
                budget=budget,
                pt_to_short=pt_to_short
            )
        }
    )

# run the simulation
simulator.run_simulation()

In [ ]:
import logging
import pandas as pd
import matplotlib.pyplot as plt
from elfpy.utils.post_processing import analysis_dict_to_dataframe
logging.getLogger().setLevel(logging.WARNING)  # events of this level and above will be tracked

trades, agent_wallets = analysis_dict_to_dataframe(simulator.analysis_dict)

spot_size = 15
spot_colors = ['blue', 'orange']
fig, ax = plt.subplots()
trades = pd.DataFrame.from_dict(simulator.analysis_dict)
x_data = trades.day
ax.scatter(x_data, trades.vault_apr, label="Vault", s=spot_size, c=spot_colors[0])
prev_apr = trades.loc[trades.run_trade_number==0].pool_apr
prev_time = 0
for day in set(x_data):
    trade_numbers = trades.loc[trades.day==day].run_trade_number
    spot_sizes = np.linspace(0.2, 0.9, len(trade_numbers))
    for trade_idx, trade_number in enumerate(trade_numbers):
        pool_apr = trades.loc[trades.run_trade_number==trade_number].pool_apr
        pool_spot_size = spot_size * 0.5 #spot_sizes[trade_idx]
        time = day + spot_sizes[trade_idx]
        if day == 0 and trade_idx == len(trade_numbers)-1:
            ax.scatter(time, pool_apr, label="Pool", s=pool_spot_size, c=spot_colors[1])
        else:
            ax.scatter(time, pool_apr, s=pool_spot_size, c=spot_colors[1])
        ax.plot([prev_time, time], [prev_apr, pool_apr], color='k', linestyle='-', linewidth=0.1)
        prev_time = time
        prev_apr = pool_apr
ax.set_xlabel("Day")
ax.set_ylabel("APR")
plt.legend()
plt.xticks([x for x in range(override_dict["num_trading_days"]+1)])
ax.set_xticklabels([str(x+1) for x in range(override_dict["num_trading_days"]+1)])
ax.set_title("Sawtooth demo")
plt.grid()

In [ ]:
trades.columns